## Extending Yaml Loading


Declaring safe_load capable object:
- Inherit from `yaml.YAMLObject`
- Set `yaml_loader = yaml.SafeLoader`

In [66]:
import yaml

class Monster(yaml.YAMLObject):
    yaml_tag = u'!Monster'
    yaml_loader = yaml.SafeLoader
    yaml_dumper = yaml.SafeDumper
    def __init__(self, name, hp, ac, attacks):
        self.name = name
        self.hp = hp
        self.ac = ac
        self.attacks = attacks
yaml_text = yaml.safe_dump(Monster(
    name='Cave lizard',
    hp=[3,6],
    ac=16,
    attacks=['BITE','HURT']))
print(yaml_text)

!Monster
ac: 16
attacks:
- BITE
- HURT
hp:
- 3
- 6
name: Cave lizard



In [67]:
yaml.safe_load(yaml_text)

### Advanced Safe Loading

If you don't want `yaml.YAMLObject` inheritance:
- Define functions for serialization.
- Register with `yaml.add_constructor` and `yaml.add_representer` for a "!Monster" tag.

You can also avoid yaml tags with `yaml.add_implicit_resolver` and regexp.

### Implicit Yaml Objects

What if you need advanced scalar construction?

``` yaml
tests:
- mark: pytest.mark.xfail
  name: feature_A_exists
- name: feature_B_exists
```

In [43]:
# First check what pytest mark is
import pytest
print(pytest.mark.xfail)
print(pytest.mark.__getattr__('xfail'))

MarkDecorator(mark=Mark(name='xfail', args=(), kwargs={}))
MarkDecorator(mark=Mark(name='xfail', args=(), kwargs={}))


In [44]:
# Now we can register constructor and representer:
import re
import yaml
import pytest

def pytest_mark_constructor(loader, node):
    value = loader.construct_scalar(node)
    return pytest.mark.__getattr__(value.rsplit(".", 1)[1])
def pytest_mark_representer(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', u'pytest.mark.%s' % data.name)
yaml.add_constructor(u'!pytest.mark', pytest_mark_constructor, Loader=yaml.SafeLoader)
yaml.add_implicit_resolver(u'!pytest.mark', re.compile(r'^pytest\.mark\.[a-zA-Z]+$'))
yaml.add_representer(type(pytest.mark.xfail), pytest_mark_representer, Dumper=yaml.SafeDumper)
# try load pytest.mark. scalar
print(yaml.safe_load('{mark: pytest.mark.xfail}'))
print(yaml.safe_load('mark: "pytest.mark.xfail"'))
# check is dumping pytest.mark. scalar works
print(yaml.safe_dump({'mark':pytest.mark.xfail}))
print ({'mark':pytest.mark.xfail})

{'mark': 'pytest.mark.xfail'}
{'mark': 'pytest.mark.xfail'}
mark: pytest.mark.xfail

{'mark': MarkDecorator(mark=Mark(name='xfail', args=(), kwargs={}))}
